In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
model_id = "RuterNorway/Llama-2-13b-chat-norwegian-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
import re
def preprocess(text):
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"(^|\n)- ", r"\1", text)
    return text

In [4]:
pipe = pipeline("text-generation", model=model, do_sample=True, tokenizer=tokenizer, max_new_tokens=512, temperature=0.7, top_p=0.95, repetition_penalty=1.15)

In [5]:
# Regular prompting
instruction = "Oppsumer følgende tekst kort"
text = """
Opposisjonen på Stortinget kom med ramsalt kritikk av NSM-saken i Stortinget i dag. Flere var også sterkt kritiske til Justisdepartementets rolle i innleggene etter Mehls redegjørelse.
– Spørsmålet er jo hvordan et så alvorlig regelbrudd kunne skje, uten at det ble fanget opp underveis, sa Fremskrittspartiets Hans Andreas Limi.

– Det er vanskelig å forstå hvordan en prosess i NSM som Justisdepartementet åpenbart var misfornøyd med, kunne fortsette uten inngripen fra departementet, fortsatte han.

Høyres Peter Frølich som leder kontrollkomiteen på Stortinget, beskrev saken som alvorlig av flere grunner:

– Saken er alvorlig for det er snakk om store summer til oppsiktsvekkende vilkår og fordi den reiser spørsmål om regjeringens etatsstyring og mest av alt fordi Grunnloven har klare regler for hvem som kan pådra staten gjeld, sa Frølich og fortsatte:

– Det vesentlige spørsmålet er om det er utvist tilstrekkelig aktsomhet, styring og kontroll fra de som sitter med ansvaret.

Også regjeringens budsjettpartner SV er kritisk:

– Hvordan er det mulig at en etat går langt utover sine fullmakter uten at noen plukker dette opp. Står vi overfor en etat som seilte sin egen sjø, eller et system som ikke har gode rutiner, en manglende kontroll? Det har vi ennå ikke fått klarhet i, sa SVs Andreas Sjalg Unneland i sitt innlegg.

Venstres Ingvild Wetrhus Thorsvik sa det slik:

– Det er ganske enkelt ufattelig at dette kunne skje. NSMs låneopptak er ikke bare et brudd på Stortingets bevilgningsreglement og Grunnloven - det er hårreisende sløsing med offentlige midler. Det er unødvendig, uansvarlig og har som konsekvens at fellesskapets midler tilfaller private.
"""  # https://www.nrk.no/nyheter/nsm-tok-opp-ulovlige-lan-1.16670810#1.16718170.1.16718170

prompt_template=f'''### Instruction: {instruction}
### Input: {preprocess(text)}
### Response:
'''
inputs = tokenizer(prompt_template, return_tensors="pt").to(0)
out = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(out[0], skip_special_tokens=True))
print(pipe(prompt_template)[0]['generated_text'][len(prompt_template):])

### Instruction: Oppsumer følgende tekst kort
### Input: Opposisjonen på Stortinget kom med ramsalt kritikk av NSM-saken i Stortinget i dag. Flere var også sterkt kritiske til Justisdepartementets rolle i innleggene etter Mehls redegjørelse. – Spørsmålet er jo hvordan et så alvorlig regelbrudd kunne skje, uten at det ble fanget opp underveis, sa Fremskrittspartiets Hans Andreas Limi. – Det er vanskelig å forstå hvordan en prosess i NSM som Justisdepartementet åpenbart var misfornøyd med, kunne fortsette uten inngripen fra departementet, fortsatte han. Høyres Peter Frølich som leder kontrollkomiteen på Stortinget, beskrev saken som alvorlig av flere grunner: – Saken er alvorlig for det er snakk om store summer til oppsiktsvekkende vilkår og fordi den reiser spørsmål om regjeringens etatsstyring og mest av alt fordi Grunnloven har klare regler for hvem som kan pådra staten gjeld, sa Frølich og fortsatte: – Det vesentlige spørsmålet er om det er utvist tilstrekkelig aktsomhet, styring og 